# LangGraph 201: Building Multi-Agent Workflows (TypeScript)


In this notebook, we're going to walk through setting up a **multi-agent workflow** in LangGraph using TypeScript and LangChain v1. We will start from a simple ReAct-style agent and add additional steps into the workflow, simulating a realistic customer support example, showcasing human-in-the-loop, long term memory, and the LangGraph pre-built library. 

The agent utilizes the [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/), and is able to handle customer inqueries related to invoice and music. 

![Arch](../images/architecture.png) 

For a deeper dive into LangGraph primitives and learning our framework, check out our [LangChain Academy](https://academy.langchain.com/courses/intro-to-langgraph)!


## Pre-work: Setup


#### Installing dependencies

First, let's install the required packages. Make sure you have Node.js installed!


In [ ]:
// Note: In a Jupyter notebook with TypeScript kernel, you would have already installed:
// npm install langchain @langchain/langgraph @langchain/core @langchain/openai @langchain/community zod sql.js

// Import required packages
import { ChatOpenAI } from "@langchain/openai";
import { StateGraph, START, END, Annotation } from "@langchain/langgraph";
import { MemorySaver, InMemoryStore } from "@langchain/langgraph";
import { tool } from "langchain";
import { z } from "zod";
import { SqlDatabase } from "@langchain/community/sql_db";
import * as https from "https";
import * as fs from "fs";
import initSqlJs from "sql.js";

console.log("Dependencies loaded successfully!");


#### Initialize the LLM

We use OpenAI in this example, but feel free to swap ChatOpenAI with other model providers that you prefer.


In [ ]:
// Initialize the model
// Make sure to set your OPENAI_API_KEY environment variable
const model = new ChatOpenAI({
  modelName: "gpt-4o",
  temperature: 0,
});

console.log("Model initialized!");


#### Loading sample customer data

The agent utilizes the [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/), which contains sample information on customer information, purchase history, and music catalog.


In [ ]:
// Function to download and setup Chinook database
async function setupChinookDatabase(): Promise<SqlDatabase> {
  const url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql";
  
  return new Promise(async (resolve, reject) => {
    https.get(url, async (response) => {
      let sqlScript = "";
      
      response.on("data", (chunk) => {
        sqlScript += chunk;
      });
      
      response.on("end", async () => {
        try {
          // Initialize sql.js
          const SQL = await initSqlJs();
          const db = new SQL.Database();
          
          // Execute the SQL script
          db.exec(sqlScript);
          
          // Create SqlDatabase wrapper
          const sqlDatabase = await SqlDatabase.fromDataSourceParams({
            appDataSource: {
              type: "sqljs",
              database: db,
            },
          });
          
          console.log("Chinook database loaded successfully!");
          resolve(sqlDatabase);
        } catch (error) {
          reject(error);
        }
      });
      
      response.on("error", (error) => {
        reject(error);
      });
    });
  });
}

// Initialize the database
const db = await setupChinookDatabase();


#### Setting up short-term and long-term memory

We will initialize a checkpointer for **short-term memory**, maintaining context within a single thread. 

**Long term memory** lets you store and recall information between conversations. We will utilize our long term memory store to store user preferences for personalization.


In [ ]:
// Initialize memory stores
const inMemoryStore = new InMemoryStore();
const checkpointer = new MemorySaver();

console.log("Memory stores initialized!");


## Part 1: Building The Sub-Agents


### 1.1 Building a ReAct Agent from Scratch

Now that we are set up, we are ready to build out our **first subagent**. This is a simple ReAct-style agent that fetches information related to music store catalog, utilizing a set of tools to generate its response. 

![react_1](../images/music_subagent.png)


#### State

**State can be thought of as the memory of the agent - it's a shared data structure that's passed on between the nodes of your graph**, representing the current snapshot of your application. 

For our customer support agent, our state will track:
1. The customer ID
2. Conversation history
3. Memory from long term memory store
4. Remaining steps (tracks # steps until it hits recursion limit)


In [ ]:
import { BaseMessage } from "@langchain/core/messages";
import { messagesStateReducer } from "@langchain/langgraph";

// Define Input State
const InputStateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
  }),
});

// Define overall State
const StateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
  }),
  customerId: Annotation<number | undefined>({
    reducer: (x, y) => y ?? x,
    default: () => undefined,
  }),
  loadedMemory: Annotation<string>({
    reducer: (x, y) => y ?? x,
    default: () => "",
  }),
  remainingSteps: Annotation<number>({
    reducer: (x, y) => y ?? x,
    default: () => 25,
  }),
});

console.log("State schemas defined!");


#### Tools

Let's define a list of **tools** our agent will have access to. Tools are functions that can act as extension of the LLM's capabilities. We will create several tools that interact with the Chinook database regarding music.

We can create tools using the `tool` decorator.


In [ ]:
// Define music catalog tools
const getAlbumsByArtist = tool(
  async ({ artist }: { artist: string }) => {
    const query = `
      SELECT Album.Title, Artist.Name 
      FROM Album 
      JOIN Artist ON Album.ArtistId = Artist.ArtistId 
      WHERE Artist.Name LIKE '%${artist}%';
    `;
    const result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_albums_by_artist",
    description: "Get albums by an artist.",
    schema: z.object({
      artist: z.string().describe("The artist name"),
    }),
  }
);

const getTracksByArtist = tool(
  async ({ artist }: { artist: string }) => {
    const query = `
      SELECT Track.Name as SongName, Artist.Name as ArtistName 
      FROM Album 
      LEFT JOIN Artist ON Album.ArtistId = Artist.ArtistId 
      LEFT JOIN Track ON Track.AlbumId = Album.AlbumId 
      WHERE Artist.Name LIKE '%${artist}%';
    `;
    const result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_tracks_by_artist",
    description: "Get songs by an artist (or similar artists).",
    schema: z.object({
      artist: z.string().describe("The artist name"),
    }),
  }
);

const getSongsByGenre = tool(
  async ({ genre }: { genre: string }) => {
    // First get genre ID
    const genreQuery = `SELECT GenreId FROM Genre WHERE Name LIKE '%${genre}%'`;
    const genreResult = await db.run(genreQuery);
    
    if (!genreResult || genreResult.length === 0) {
      return `No songs found for the genre: ${genre}`;
    }
    
    const genreIds = genreResult.map((row: any) => row.GenreId).join(", ");
    
    const songsQuery = `
      SELECT Track.Name as SongName, Artist.Name as ArtistName
      FROM Track
      LEFT JOIN Album ON Track.AlbumId = Album.AlbumId
      LEFT JOIN Artist ON Album.ArtistId = Artist.ArtistId
      WHERE Track.GenreId IN (${genreIds})
      GROUP BY Artist.Name
      LIMIT 8;
    `;
    
    const songs = await db.run(songsQuery);
    return JSON.stringify(songs);
  },
  {
    name: "get_songs_by_genre",
    description: "Fetch songs from the database that match a specific genre.",
    schema: z.object({
      genre: z.string().describe("The genre of the songs to fetch"),
    }),
  }
);

const checkForSongs = tool(
  async ({ songTitle }: { songTitle: string }) => {
    const query = `SELECT * FROM Track WHERE Name LIKE '%${songTitle}%';`;
    const result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "check_for_songs",
    description: "Check if a song exists by its name.",
    schema: z.object({
      songTitle: z.string().describe("The song title to search for"),
    }),
  }
);

const musicTools = [getAlbumsByArtist, getTracksByArtist, getSongsByGenre, checkForSongs];
const llmWithMusicTools = model.bindTools(musicTools);

console.log("Music tools defined!");


In [ ]:
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { SystemMessage, HumanMessage } from "@langchain/core/messages";

// Create tool node
const musicToolNode = new ToolNode(musicTools);

// Helper function to generate music assistant prompt
function generateMusicAssistantPrompt(memory: string = "None"): string {
  return `
<important_background>
You are a member of the assistant team, your role specifically is to focused on helping customers discover and learn about music in our digital catalog. 
If you are unable to find playlists, songs, or albums associated with an artist, it is okay. 
Just respond that the catalog does not have any playlists, songs, or albums associated with that artist.
You also have context on any saved user preferences, helping you to tailor your response. 
IMPORTANT: Your interaction with the customer is done through an automated system. You are not directly interacting with the customer, so avoid chitchat or follow up questions and focus PURELY on responding to the request with the necessary information. 
</important_background>

<core_responsibilities>
- Search and provide accurate information about songs, albums, artists, and playlists
- Offer relevant recommendations based on customer interests
- Handle music-related queries with attention to detail
- Help customers discover new music they might enjoy
- You are routed only when there are questions related to music catalog; ignore other questions. 
</core_responsibilities>

<guidelines>
1. Always perform thorough searches before concluding something is unavailable
2. If exact matches aren't found, try:
   - Checking for alternative spellings
   - Looking for similar artist names
   - Searching by partial matches
   - Checking different versions/remixes
3. When providing song lists:
   - Include the artist name with each song
   - Mention the album when relevant
   - Note if it's part of any playlists
   - Indicate if there are multiple versions
</guidelines>

Additional context is provided below: 

Prior saved user preferences: ${memory}

Message history is also attached.  
`;
}

// Music assistant node
async function musicAssistant(state: typeof StateAnnotation.State) {
  // Fetching long term memory
  let memory = "None";
  if (state.loadedMemory) {
    memory = state.loadedMemory;
  }

  // Instructions for our agent
  const musicAssistantPrompt = generateMusicAssistantPrompt(memory);

  // Invoke the model
  const response = await llmWithMusicTools.invoke([
    new SystemMessage(musicAssistantPrompt),
    ...state.messages,
  ]);

  // Update the state
  return { messages: [response] };
}

console.log("Nodes defined!");


In [ ]:
import { AIMessage } from "@langchain/core/messages";

// Conditional edge that determines whether to continue or not
function shouldContinue(state: typeof StateAnnotation.State): "continue" | "end" {
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1] as AIMessage;

  // If there is no function call, then we finish
  if (!lastMessage.tool_calls || lastMessage.tool_calls.length === 0) {
    return "end";
  }
  // Otherwise if there is, we continue
  return "continue";
}

console.log("Conditional edge defined!");


#### Compile Graph!

Now that we've defined our State and Nodes, let's put it all together and construct our react agent!


In [ ]:
// Create the workflow
const musicWorkflow = new StateGraph(StateAnnotation)
  // Add nodes
  .addNode("music_assistant", musicAssistant)
  .addNode("music_tool_node", musicToolNode)
  // Add edges
  .addEdge(START, "music_assistant")
  .addConditionalEdges("music_assistant", shouldContinue, {
    continue: "music_tool_node",
    end: END,
  })
  .addEdge("music_tool_node", "music_assistant");

// Compile the graph
const musicCatalogSubagent = musicWorkflow.compile({
  checkpointer,
  store: inMemoryStore,
});

console.log("Music catalog subagent compiled!");


#### Testing

Let's see how it works!


In [ ]:
import { v4 as uuidv4 } from "uuid";

const threadId = uuidv4();
const question = "I like the Rolling Stones. What songs do you recommend by them or by other artists that I might like?";
const config = { configurable: { thread_id: threadId } };

const result = await musicCatalogSubagent.invoke(
  { messages: [new HumanMessage(question)] },
  config
);

// Print all messages
for (const message of result.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
  if (message._getType() === "ai" && (message as AIMessage).tool_calls) {
    console.log("Tool calls:", (message as AIMessage).tool_calls);
  }
}


### 1.2. Building ReAct Agent using LangChain's `createAgent()`

LangChain offers a powerful ReAct agent architecture out of the box, allowing us to quickly create and iterate on applications that leverage this widespread design. More information of this pre-built architecture can be found [here](https://docs.langchain.com/oss/javascript/releases/langchain-v1#createagent)

In the last workflow, we have seen how we can build a ReAct agent from scratch. Now, we will show how we can leverage the LangChain pre-built ReAct agent to achieve similar results. 

![react_2](../images/invoice_subagent.png)

Our **invoice info subagent** is responsible for all customer queries related to the invoices.


#### Defining tools and prompt

Similarly, let's first define a set of tools and our agent prompt below. 

Here, we will utilize `InjectedState`, an annotation for injecting graph state into tool arguments. This annotation enables tools to access graph state without exposing state management details to the language model.


In [ ]:
import { InjectedState } from "@langchain/core/tools";

// Invoice tools with injected state
const getInvoicesByCustomerSortedByDate = tool(
  async ({ customerId }: { customerId: number }) => {
    const query = `SELECT * FROM Invoice WHERE CustomerId = ${customerId} ORDER BY InvoiceDate DESC;`;
    const result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_invoices_by_customer_sorted_by_date",
    description: "Look up all invoices for a customer using their ID. The customer ID is in a state variable, so you will not see it in the message history. The invoices are sorted in descending order by invoice date.",
    schema: z.object({
      customerId: z.number().describe("The customer ID from state"),
    }),
    injectedState: {
      customerId: InjectedState("customerId"),
    },
  }
);

const getInvoicesSortedByUnitPrice = tool(
  async ({ customerId }: { customerId: number }) => {
    const query = `
      SELECT Invoice.*, InvoiceLine.UnitPrice
      FROM Invoice
      JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
      WHERE Invoice.CustomerId = ${customerId}
      ORDER BY InvoiceLine.UnitPrice DESC;
    `;
    const result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_invoices_sorted_by_unit_price",
    description: "Use this tool when the customer wants to know the details of one of their invoices based on the unit price/cost. This tool looks up all invoices for a customer, and sorts the unit price from highest to lowest. The customer ID is in a state variable.",
    schema: z.object({
      customerId: z.number().describe("The customer ID from state"),
    }),
    injectedState: {
      customerId: InjectedState("customerId"),
    },
  }
);

const getEmployeeByInvoiceAndCustomer = tool(
  async ({ invoiceId, customerId }: { invoiceId: number; customerId: number }) => {
    const query = `
      SELECT Employee.FirstName, Employee.Title, Employee.Email
      FROM Employee
      JOIN Customer ON Customer.SupportRepId = Employee.EmployeeId
      JOIN Invoice ON Invoice.CustomerId = Customer.CustomerId
      WHERE Invoice.InvoiceId = ${invoiceId} AND Invoice.CustomerId = ${customerId};
    `;
    const result = await db.run(query);
    
    if (!result || result.length === 0) {
      return `No employee found for invoice ID ${invoiceId} and customer identifier ${customerId}.`;
    }
    return JSON.stringify(result);
  },
  {
    name: "get_employee_by_invoice_and_customer",
    description: "This tool will take in an invoice ID and return the employee information associated with the invoice. The customer ID is in a state variable.",
    schema: z.object({
      invoiceId: z.number().describe("The ID of the specific invoice"),
      customerId: z.number().describe("The customer ID from state"),
    }),
    injectedState: {
      customerId: InjectedState("customerId"),
    },
  }
);

const invoiceTools = [
  getInvoicesByCustomerSortedByDate,
  getInvoicesSortedByUnitPrice,
  getEmployeeByInvoiceAndCustomer,
];

console.log("Invoice tools defined!");


In [ ]:
const invoiceSubagentPrompt = `
<important_background>
You are a subagent among a team of assistants. You are specialized for retrieving and processing invoice information. 
Invoices contain information such as song purchases and billing history. Only respond to questions if they relate in some way to billing, invoices, or purchases.  
If you are unable to retrieve the invoice information, respond that you are unable to retrieve the information.
IMPORTANT: Your interaction with the customer is done through an automated system. You are not directly interacting with the customer, so avoid chitchat or follow up questions and focus PURELY on responding to the request with the necessary information. 
</important_background>
 
<tools>
You have access to three tools. These tools enable you to retrieve and process invoice information from the database. Here are the tools:
- get_invoices_by_customer_sorted_by_date: This tool retrieves all invoices for a customer, sorted by invoice date. 
- get_invoices_sorted_by_unit_price: This tool retrieves all invoices for a customer, sorted by unit price.
- get_employee_by_invoice_and_customer: This tool retrieves the employee information associated with an invoice and a customer.
</tools>

<core_responsibilities>
- Retrieve and process invoice information from the database
- Provide detailed information about invoices, including customer details, invoice dates, total amounts, employees associated with the invoice, etc. when the customer asks for it.
- Always maintain a professional, friendly, and patient demeanor in your responses.
</core_responsibilities>

You may have additional context that you should use to help answer the customer's query. It will be provided to you below:
`;

console.log("Invoice prompt defined!");


In [ ]:
import { createAgent } from "langchain";

// Define the subagent using LangChain v1's createAgent
const invoiceInformationSubagent = createAgent({
  model,
  tools: invoiceTools,
  systemPrompt: invoiceSubagentPrompt,
  stateSchema: StateAnnotation,
  checkpointer,
  store: inMemoryStore,
});

console.log("Invoice information subagent created!");


#### Testing!

Let's try our new agent out!


In [ ]:
const threadId2 = uuidv4();
const question2 = "What was my most recent invoice, and who was the employee that helped me with it?";
const config2 = { configurable: { thread_id: threadId2 } };

const result2 = await invoiceInformationSubagent.invoke(
  { 
    messages: [new HumanMessage(question2)],
    customerId: 1 
  },
  config2
);

// Print all messages
for (const message of result2.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
  if (message._getType() === "ai" && (message as AIMessage).tool_calls) {
    console.log("Tool calls:", (message as AIMessage).tool_calls);
  }
}


#### Part 2.1.1 Writing the supervisor's prompt


In [ ]:
const supervisorPrompt = `
<background>
You are an expert customer support assistant for a digital music store. You can handle music catalog or invoice related question regarding past purchases, song or album availabilities. 
You are dedicated to providing exceptional service and ensuring customer queries are answered thoroughly, and have a team of subagents that you can use to help answer queries from customers. 
Your primary role is to delegate tasks to this multi-agent team in order to answer queries from customers. 
</background>

<important_instructions>
Always respond to the customer through summarizing the findings of the individual responses from subagents. 
If a question is unrelated to music or invoice, politely remind the customer regarding your scope of work. Do not answer unrelated answers.
Based on the existing steps that have been taken in the messages, your role is to call the appropriate subagent based on the users query.
</important_instructions>

<tools>
You have 2 tools available to delegate to the subagents on your team:
1. music_catalog_information_subagent: Call this tool to delegate to the music subagent. The music agent has access to user's saved music preferences. It can also retrieve information about the digital music store's music 
catalog (albums, tracks, songs, etc.) from the database. 
2. invoice_information_subagent: Call this tool to delegate to the invoice subagent. This subagent is able to retrieve information about a customer's past purchases or invoices 
from the database. 
</tools>
`;

console.log("Supervisor prompt defined!");


In [ ]:
// Create supervisor tools that delegate to subagents
const callInvoiceInformationSubagent = tool(
  async ({ query, customerId }: { query: string; customerId: number }) => {
    const result = await invoiceInformationSubagent.invoke({
      messages: [{ role: "user", content: query }],
      customerId,
    });
    const subagentResponse = result.messages[result.messages.length - 1].content;
    return subagentResponse;
  },
  {
    name: "invoice_information_subagent",
    description: "An agent that can assistant with all invoice-related queries. It can retrieve information about a customers past purchases or invoices.",
    schema: z.object({
      query: z.string().describe("The query to send to the invoice subagent"),
      customerId: z.number().describe("The customer ID from state"),
    }),
    injectedState: {
      customerId: InjectedState("customerId"),
    },
  }
);

const callMusicCatalogSubagent = tool(
  async ({ query }: { query: string }) => {
    const result = await musicCatalogSubagent.invoke({
      messages: [{ role: "user", content: query }],
    });
    const subagentResponse = result.messages[result.messages.length - 1].content;
    return subagentResponse;
  },
  {
    name: "music_catalog_subagent",
    description: "An agent that can assistant with all music-related queries. This agent has access to user's saved music preferences. It can also retrieve information about the digital music store's music catalog (albums, tracks, songs, etc.) from the database.",
    schema: z.object({
      query: z.string().describe("The query to send to the music catalog subagent"),
    }),
  }
);

console.log("Supervisor tools defined!");


In [ ]:
// Create the supervisor agent
const supervisor = createAgent({
  model,
  tools: [callInvoiceInformationSubagent, callMusicCatalogSubagent],
  systemPrompt: supervisorPrompt,
  stateSchema: StateAnnotation,
  checkpointer,
  store: inMemoryStore,
});

console.log("Supervisor agent created!");


In [ ]:
const threadId3 = uuidv4();
const question3 = "how much was my most recent purchase and what albums do you have by the rolling stones?";
const config3 = { configurable: { thread_id: threadId3 } };

const result3 = await supervisor.invoke(
  { 
    messages: [new HumanMessage(question3)],
    customerId: 1 
  },
  config3
);

// Print all messages
for (const message of result3.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
  if (message._getType() === "ai" && (message as AIMessage).tool_calls) {
    console.log("Tool calls:", (message as AIMessage).tool_calls);
  }
}


In [ ]:
// Schema for parsing user-provided account information
const UserInputSchema = z.object({
  identifier: z.string().describe("Identifier, which can be a customer ID, email, or phone number."),
});

const structuredLlm = model.withStructuredOutput(UserInputSchema);

const structuredSystemPrompt = `You are a customer service representative responsible for extracting customer identifier.
Only extract the customer's account information from the message history. 
If they haven't provided the information yet, return an empty string for the identifier`;

console.log("Structured extraction setup complete!");


In [ ]:
// Helper function to get customer ID from identifier
async function getCustomerIdFromIdentifier(identifier: string): Promise<number | null> {
  // Check if it's a digit (customer ID)
  if (/^\d+$/.test(identifier)) {
    return parseInt(identifier);
  }
  // Check if it's a phone number (starts with +)
  else if (identifier.startsWith("+")) {
    const query = `SELECT CustomerId FROM Customer WHERE Phone = '${identifier}';`;
    const result = await db.run(query);
    if (result && result.length > 0) {
      return result[0].CustomerId;
    }
  }
  // Check if it's an email (contains @)
  else if (identifier.includes("@")) {
    const query = `SELECT CustomerId FROM Customer WHERE Email = '${identifier}';`;
    const result = await db.run(query);
    if (result && result.length > 0) {
      return result[0].CustomerId;
    }
  }
  return null;
}

console.log("Helper function defined!");


In [ ]:
// Verify info node
async function verifyInfo(state: typeof StateAnnotation.State) {
  if (state.customerId === undefined) {
    const systemInstructions = `
You are a music store agent, where you are trying to verify the customer identity as the first step of the customer support process. 
You cannot support them until their account is verified. 
In order to verify their identity, one of their customer ID, email, or phone number needs to be provided.
If the customer has not provided their identifier, please ask them for it.
If they have provided the identifier but cannot be found, please ask them to revise it.

IMPORTANT: Do NOT ask any questions about their request, or make any attempt at addressing their request until their identity is verified. It is CRITICAL that you only ask about their identity for security purposes.
`;

    const userInput = state.messages[state.messages.length - 1];
    
    // Parse for customer ID
    const parsedInfo = await structuredLlm.invoke([
      new SystemMessage(structuredSystemPrompt),
      userInput,
    ]);
    
    // Extract details
    const identifier = parsedInfo.identifier;
    
    let customerId: number | null = null;
    // Attempt to find the customer ID
    if (identifier) {
      customerId = await getCustomerIdFromIdentifier(identifier);
    }
    
    if (customerId !== null) {
      const intentMessage = new AIMessage(
        `Thank you for providing your information! I was able to verify your account with customer id ${customerId}.`
      );
      return {
        customerId: customerId,
        messages: [intentMessage],
      };
    } else {
      const response = await model.invoke([
        new SystemMessage(systemInstructions),
        ...state.messages,
      ]);
      return { messages: [response] };
    }
  } else {
    // Customer ID already exists, pass through
    return {};
  }
}

console.log("Verify info node defined!");


In [ ]:
import { interrupt } from "@langchain/langgraph";

// Human input node
function humanInput(state: typeof StateAnnotation.State) {
  const userInput = interrupt("Please provide input.");
  return { messages: [new HumanMessage(userInput as string)] };
}

console.log("Human input node defined!");


In [ ]:
// Conditional edge
function shouldInterrupt(state: typeof StateAnnotation.State): "continue" | "interrupt" {
  if (state.customerId !== undefined) {
    return "continue";
  } else {
    return "interrupt";
  }
}

// Build the graph with human-in-the-loop
const multiAgentVerify = new StateGraph(StateAnnotation, {
  input: InputStateAnnotation,
})
  .addNode("verify_info", verifyInfo)
  .addNode("human_input", humanInput)
  .addNode("supervisor", supervisor)
  .addEdge(START, "verify_info")
  .addConditionalEdges("verify_info", shouldInterrupt, {
    continue: "supervisor",
    interrupt: "human_input",
  })
  .addEdge("human_input", "verify_info")
  .addEdge("supervisor", END);

const multiAgentVerifyGraph = multiAgentVerify.compile({
  checkpointer,
  store: inMemoryStore,
});

console.log("Multi-agent with verification graph compiled!");


In [ ]:
const threadId4 = uuidv4();
const question4 = "How much was my most recent purchase?";
const config4 = { configurable: { thread_id: threadId4 } };

const result4 = await multiAgentVerifyGraph.invoke(
  { messages: [new HumanMessage(question4)] },
  config4
);

// Print all messages
for (const message of result4.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


In [ ]:
import { Command } from "@langchain/langgraph";

// Resume from interrupt
const question5 = "My phone number is +55 (12) 3923-5555.";
const result5 = await multiAgentVerifyGraph.invoke(
  Command.resume(question5),
  config4
);

// Print all messages
for (const message of result5.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


In [ ]:
const question6 = "What albums do you have by U2?";
const result6 = await multiAgentVerifyGraph.invoke(
  { messages: [new HumanMessage(question6)] },
  config4
);

// Print all messages
for (const message of result6.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


In [ ]:
// User profile structure for creating memory
const UserProfileSchema = z.object({
  customerId: z.string().describe("The customer ID of the customer"),
  musicPreferences: z.array(z.string()).describe("The music preferences of the customer"),
});

// Helper function to format user memory
function formatUserMemory(userData: any): string {
  const profile = userData.memory;
  let result = "";
  if (profile && profile.musicPreferences && profile.musicPreferences.length > 0) {
    result += `Music Preferences: ${profile.musicPreferences.join(", ")}`;
  }
  return result.trim();
}

// Load memory node
async function loadMemory(state: typeof StateAnnotation.State) {
  const userId = state.customerId?.toString();
  if (!userId) {
    return { loadedMemory: "" };
  }
  
  const namespace = ["memory_profile", userId];
  const existingMemory = await inMemoryStore.get(namespace, "user_memory");
  let formattedMemory = "";
  
  if (existingMemory && existingMemory.value) {
    formattedMemory = formatUserMemory(existingMemory.value);
  }
  
  return { loadedMemory: formattedMemory };
}

console.log("Load memory node defined!");


In [ ]:
const createMemoryPrompt = `You are an expert analyst that is observing a conversation that has taken place between a customer and a customer support assistant. The customer support assistant works for a digital music store, and has utilized a multi-agent team to answer the customer's request. 
You are tasked with analyzing the conversation that has taken place between the customer and the customer support assistant, and updating the memory profile associated with the customer. 
You specifically care about saving any music interest the customer has shared about themselves, particularly their music preferences to their memory profile.

<core_instructions>
1. The memory profile may be empty. If it's empty, you should ALWAYS create a new memory profile for the customer.
2. You should identify any music interest the customer during the conversation and add it to the memory profile **IF** it is not already present.
3. For each key in the memory profile, if there is no new information, do NOT update the value - keep the existing value unchanged.
4. ONLY update the values in the memory profile if there is new information.
</core_instructions>

<expected_format>
The customer's memory profile should have the following fields:
- customerId: the customer ID of the customer
- musicPreferences: the music preferences of the customer

IMPORTANT: ENSURE your response is an object with these fields.
</expected_format>

<important_context>
**IMPORTANT CONTEXT BELOW**
To help you with this task, I have attached the conversation that has taken place between the customer and the customer support assistant below, as well as the existing memory profile associated with the customer that you should either update or create. 

The conversation between the customer and the customer support assistant that you should analyze is as follows:
{conversation}

The existing memory profile associated with the customer that you should either update or create based on the conversation is as follows:
{memory_profile}

</important_context>

Reminder: Take a deep breath and think carefully before responding.
`;

// Create memory node
async function createMemory(state: typeof StateAnnotation.State) {
  const userId = state.customerId?.toString();
  if (!userId) {
    return {};
  }
  
  const namespace = ["memory_profile", userId];
  const formattedMemory = state.loadedMemory || "";
  
  const formattedSystemMessage = new SystemMessage(
    createMemoryPrompt
      .replace("{conversation}", JSON.stringify(state.messages))
      .replace("{memory_profile}", formattedMemory)
  );
  
  const updatedMemory = await model.withStructuredOutput(UserProfileSchema).invoke([formattedSystemMessage]);
  
  const key = "user_memory";
  await inMemoryStore.put(namespace, key, { memory: updatedMemory });
  
  return {};
}

console.log("Create memory node defined!");


In [ ]:
// Build the final multi-agent graph with memory
const multiAgentFinal = new StateGraph(StateAnnotation, {
  input: InputStateAnnotation,
})
  .addNode("verify_info", verifyInfo)
  .addNode("human_input", humanInput)
  .addNode("load_memory", loadMemory)
  .addNode("supervisor", supervisor)
  .addNode("create_memory", createMemory)
  .addEdge(START, "verify_info")
  .addConditionalEdges("verify_info", shouldInterrupt, {
    continue: "load_memory",
    interrupt: "human_input",
  })
  .addEdge("human_input", "verify_info")
  .addEdge("load_memory", "supervisor")
  .addEdge("supervisor", "create_memory")
  .addEdge("create_memory", END);

const multiAgentFinalGraph = multiAgentFinal.compile({
  checkpointer,
  store: inMemoryStore,
});

console.log("Final multi-agent graph with memory compiled!");


In [ ]:
const threadId7 = uuidv4();
const question7 = "My phone number is +55 (12) 3923-5555. How much was my most recent purchase? What albums do you have by the Rolling Stones?";
const config7 = { configurable: { thread_id: threadId7 } };

const result7 = await multiAgentFinalGraph.invoke(
  { messages: [new HumanMessage(question7)] },
  config7
);

// Print all messages
for (const message of result7.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


In [ ]:
const userId = "1";
const namespace = ["memory_profile", userId];
const memory = await inMemoryStore.get(namespace, "user_memory");

if (memory && memory.value) {
  const savedMusicPreferences = memory.value.memory.musicPreferences;
  console.log("Saved music preferences:", savedMusicPreferences);
}
